In [1]:
import time
import torch
import numpy as np
import pandas as pd
from sklearn.model_selection import ParameterGrid
from heat import heat_nn

In [2]:
layers = []
for i in range(2, 5):
    for params in ParameterGrid({str(j): [50, 100, 200, 300] for j in range(i)}):
        layers.append([params[str(j)] for j in range(i)] + [1])

param_grid = {
    "layers": layers,
    "lr": [1e-3, 1e-4],
    "data points x": [10, 50, 100],
    "data points t": [10, 50, 100],
}

In [ ]:
results = pd.DataFrame(columns=["params", "epochs", "loss", "mse"])
start_time = time.time()

for idx, params in enumerate(ParameterGrid(param_grid)):
    actications = [torch.tanh for _ in range(len(params["layers"])-1)] + [None]
    model = heat_nn(params["layers"], actications, 1, lambda x: torch.sin(torch.pi * x))
    model.set_data(params["data points x"], params["data points t"])
    model.set_analytic_solution(lambda x, t: np.sin(np.pi * x) * np.exp(-np.pi**2 * t))
 
    for i in range(30):
        model.train(lr=params["lr"], weight_decay=0, epochs=100)
        loss = model.loss_fn()
        mse = model.mse()
        results = pd.concat([results, pd.DataFrame([{
            "params": params,
            "epochs": (i+1)*100,
            "loss": loss,
            "mse": mse
        }])], ignore_index=True)
    
    t = int(time.time() - start_time)
    print(f"Completed {idx+1} out of {len(ParameterGrid(param_grid))} configurations. Time elapsed: {t//3600}h {(t%3600)//60}m {t%60}s")


/tmp/ipykernel_14416/3555072615.py:8: DeprecationWarning: __array_wrap__ must accept context and return_scalar arguments (positionally) in the future. (Deprecated NumPy 2.0)
  model.set_analytic_solution(lambda x, t: np.sin(np.pi * x) * np.exp(-np.pi**2 * t))
/tmp/ipykernel_14416/3555072615.py:14: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results = pd.concat([results, pd.DataFrame([{


Completed 1 out of 6048 configurations. Time elapsed: 0h 0m 23s
Completed 2 out of 6048 configurations. Time elapsed: 0h 0m 39s
Completed 3 out of 6048 configurations. Time elapsed: 0h 0m 56s
Completed 4 out of 6048 configurations. Time elapsed: 0h 1m 11s
Completed 5 out of 6048 configurations. Time elapsed: 0h 1m 29s
Completed 6 out of 6048 configurations. Time elapsed: 0h 1m 48s
Completed 7 out of 6048 configurations. Time elapsed: 0h 2m 7s
Completed 8 out of 6048 configurations. Time elapsed: 0h 2m 26s
Completed 9 out of 6048 configurations. Time elapsed: 0h 2m 43s
Completed 10 out of 6048 configurations. Time elapsed: 0h 2m 58s
Completed 11 out of 6048 configurations. Time elapsed: 0h 3m 19s
Completed 12 out of 6048 configurations. Time elapsed: 0h 3m 36s
Completed 13 out of 6048 configurations. Time elapsed: 0h 3m 54s
Completed 14 out of 6048 configurations. Time elapsed: 0h 4m 14s
Completed 15 out of 6048 configurations. Time elapsed: 0h 4m 34s
Completed 16 out of 6048 configurat

In [ ]:
with open("heat_nn_hyperparameter_tuning_results.pkl", "wb") as f:
    pd.to_pickle(results, f)